**9주차 실습 - 토픽 모델링**

# 1. 20 뉴스그룹 데이터 준비

- 데이터셋 설명: http://qwone.com/~jason/20Newsgroups/
- scikit-learn 라이브러리: http://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

In [16]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space',
              'comp.sys.ibm.pc.hardware', 'sci.crypt']

# 20 Newsgroups 데이터셋 로드
newsgroups = fetch_20newsgroups(categories=categories, shuffle=True, random_state=42,
                                remove=('headers', 'footers', 'quotes'))

In [ ]:
newsgroups.target_names

In [ ]:
len(newsgroups.data)

# 2. TF-IDF 행렬 변환

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vectorizer 생성
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5, max_features=1000)

# 문서를 TF-IDF 행렬로 변환
tfidf_matrix = vectorizer.fit_transform(newsgroups.data)

In [ ]:
tfidf_matrix.shape

In [ ]:
tfidf_matrix[0]

# 3. LSA 실행

In [22]:
from sklearn.decomposition import TruncatedSVD

num_topics = 6  # 토픽 수

# Truncated SVD (LSA) 수행
lsa = TruncatedSVD(n_components=num_topics, random_state=42)
lsa_matrix = lsa.fit_transform(tfidf_matrix)

# 단어 feature
terms = vectorizer.get_feature_names_out()

In [ ]:
lsa_matrix.shape

In [ ]:
lsa_matrix[0]

In [ ]:
# 각 토픽에 대한 단어 출력
def print_top_terms(lsa, terms, n_words=10):
    for i, topic in enumerate(lsa.components_):
        # 토픽 내 단어의 중요도 순으로 정렬된 인덱스를 가져옴
        top_terms_idx = topic.argsort()[-n_words:][::-1]
        # 인덱스를 사용하여 실제 단어를 찾아냄
        top_terms = [terms[idx] for idx in top_terms_idx]
        print(f"토픽 {i + 1}: {' '.join(top_terms)}")

print('# LSA 결과:')
print_top_terms(lsa, terms, 10)

# 4. LDA 실행

In [26]:
from sklearn.decomposition import LatentDirichletAllocation

# LDA 수행
lda = LatentDirichletAllocation(
    n_components=num_topics,  # 토픽의 수
    max_iter=5,  # 알고리즘 최대 반복 횟수(수렴할 때까지 또는 주어진 반복 횟수까지)
    topic_word_prior=0.1,  # 토픽 내에서 단어의 확률 분포를 제어하는 매개변수(값이 작을수록 토픽 내에서 단어가 더 다양)
    doc_topic_prior=1.0,  # 문서 내 토픽의 분포를 제어하는 매개변수(작을수록 문서가 하나의 토픽에 집중)
    learning_method='online',
    n_jobs=-1,
    random_state=0
)

lda_matrix = lda.fit_transform(tfidf_matrix)



In [ ]:
lda_matrix.shape

In [ ]:
lda_matrix[0]

In [ ]:
print('# LDA 결과:')
print_top_terms(lda, terms, 10)

# 5. 토픽 할당

In [ ]:
# LSA, LDA를 기반으로 문서에 토픽 할당
def assign_topics(topic_matrix, num_topics):
    return np.argmax(topic_matrix, axis=1) + 1

newsgroups['lsa_topic'] = assign_topics(lsa_matrix, num_topics)
newsgroups['lda_topic'] = assign_topics(lda_matrix, num_topics)

df = pd.DataFrame({'문서': newsgroups.data[:10], 'LSA 토픽': newsgroups['lsa_topic'][:10], 'LDA 토픽': newsgroups['lda_topic'][:10]})
df